In [157]:
import json
from boto3.dynamodb.types import TypeDeserializer

def ddb_deserialize(r, type_deserializer = TypeDeserializer()):
  return type_deserializer.deserialize({"M": r})

In [160]:
import boto3
import pandas

client = boto3.client('dynamodb')
items = []
page = True
query= """SELECT * FROM "Session-e3tq3jp6jjh4xjjpvydbqj64e4-dev" """

resp = client.execute_statement(Statement=query, Limit=1000)
items.extend(resp['Items'])
while 'LastEvaluatedKey' in resp:
  resp = client.execute_statement(Statement=query,Limit=1000,NextToken=resp["NextToken"])
  items.extend(resp['Items'])

items = [ ddb_deserialize(r) for r in items ]

df = pandas.DataFrame(items)

df

,__typename,_lastChangedAt,linearAccerationTimestamp,createdAt,name,quaternionW,quaternionX,linearAccerationZ,quaternionY,linearAccerationY,quaternionZ,_version,linearAccerationX,updatedAt,owner,id,quaternionTimestamp
0,Session,1671822976889,"[1671822898.4810004, 1671822898.4910004, 16718...",2022-12-23T19:16:16.815Z,Squats 1,"[0.2219315618276596, 0.2219315618276596, 0.221...","[-0.025271181017160416, -0.025271181017160416,...","[0.003181945299729705, 0.005135068204253912, 0...","[0.030318086966872215, 0.030318086966872215, 0...","[0.005969487130641937, 0.006946048699319363, 0...","[0.974263072013855, 0.974263072013855, 0.97426...",1,"[0.014128324575722218, 0.013640043325722218, 0...",2022-12-23T19:16:16.815Z,43d1cc9a-eacd-4d09-8c12-7352818673de::jdavis,5cf3939f-06d9-4aa5-80ff-05ad0ed47b98,"[1671822898.4810004, 1671822898.4910004, 16718..."
1,Session,1671851399908,"[1671851259.8320007, 1671851259.843001, 167185...",2022-12-24T03:09:59.897Z,2,"[0.9991313815116882, 0.9991336464881897, 0.999...","[0.023245390504598618, 0.023248231038451195, 0...","[-0.001422734116204083, 0.0024776773061603308,...","[0.02163386158645153, 0.021528679877519608, 0....","[0.0031633861362934113, 0.001210259972140193, ...","[0.02698240801692009, 0.026979960501194, 0.026...",1,"[0.0009600811172276735, -0.0012031638761982322...",2022-12-24T03:09:59.897Z,43d1cc9a-eacd-4d09-8c12-7352818673de::jdavis,7c35bdaf-60e6-4f90-9d3c-80311acaea42,"[1671851259.8320007, 1671851259.843001, 167185..."
2,Session,1671851245308,"[1671851167.6890006, 1671851167.7000008, 16718...",2022-12-24T03:07:25.274Z,First,"[0.999974250793457, 0.999974250793457, 0.99997...","[0.004534398205578327, 0.004534398205578327, 0...","[0.0034936240408569574, 0.0025170627050101757,...","[0.0055681513622403145, 0.0055681513622403145,...","[0.006068170070648193, 0.008021295070648193, 0...","[-0.0000013438473160931608, -0.000001343847316...",1,"[-0.010836626403033733, -0.012789752334356308,...",2022-12-24T03:07:25.274Z,43d1cc9a-eacd-4d09-8c12-7352818673de::jdavis,ce963746-fea5-4715-800d-37b438b5beb2,"[1671851167.6890006, 1671851167.7000008, 16718..."
3,Session,1671823126602,"[1671822934.3080006, 1671822934.3180003, 16718...",2022-12-23T19:18:46.528Z,yeet,"[0.9407093524932861, 0.9406892657279968, 0.940...","[0.307903915643692, 0.30795177817344666, 0.307...","[0.008580555208027363, 0.009934778325259686, 0...","[0.13391804695129395, 0.1338629424571991, 0.13...","[-0.004139835014939308, -0.0015515387058258057...","[-0.04823977127671242, -0.048479072749614716, ...",1,"[-0.01074779313057661, -0.008263429626822472, ...",2022-12-23T19:18:46.528Z,43d1cc9a-eacd-4d09-8c12-7352818673de::jdavis,f546e387-899b-476a-a993-1543c4710392,"[1671822934.3080006, 1671822934.3180003, 16718..."
4,Session,1672076069786,"[1672073388.025, 1672073388.035, 1672073388.04...",2022-12-26T17:34:29.733Z,3 rotations 2,"[0.9468138813972473, 0.9468017220497131, 0.946...","[-0.29967445135116577, -0.29988008737564087, -...","[-0.01034603826701641, -0.013361495919525623, ...","[-0.09628232568502426, -0.09541510790586472, -...","[0.023349663242697716, -0.00700776744633913, 0...","[-0.06684619933366776, -0.06733762472867966, -...",1,"[0.013941043056547642, 0.002568020485341549, -...",2022-12-26T17:34:29.733Z,43d1cc9a-eacd-4d09-8c12-7352818673de::jdavis,a55eb931-6161-46f3-9a63-8bdb4bb82b66,"[1672073388.025, 1672073388.035, 1672073388.04..."
5,Session,1672076074989,"[1672073337.7350006, 1672073337.7460008, 16720...",2022-12-26T17:34:34.938Z,3 rotations,"[0.9656655192375183, 0.9655967950820923, 0.965...","[-0.25631627440452576, -0.25653955340385437, -...","[0.04639124497771263, 0.04127484932541847, 0.0...","[-0.0410739928483963, -0.04122942313551903, -0...","[-0.0019042075145989656, -0.004950540605932474...","[-0.01025026012212038, -0.010506663471460342, ...",1,"[0.01649211160838604, 0.02973334863781929, 0.0...",2022-12-26T17:34:34.938Z,43d1cc9a-eacd-4d09-8c12-7352818673de::jdavis,53178433-928e-4343-9d9e-4f73db29b702,"[1672073337.7350006, 16720733